In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('/content/drive/MyDrive/FastRcnn')
!pwd

/content/drive/MyDrive/FastRcnn


In [4]:
!git clone https://github.com/zjZSTU/Fast-R-CNN.git

Cloning into 'Fast-R-CNN'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 85 (delta 34), reused 76 (delta 25), pack-reused 0
Unpacking objects: 100% (85/85), done.


multi_task_loss

In [ ]:
# -*- coding: utf-8 -*-

"""
@date: 2020/3/31 下午3:33
@file: multi_task_loss.py
@author: zj
@description: 
"""

import torch
import torch.nn as nn

from models.smooth_l1_loss import SmoothL1Loss


class MultiTaskLoss(nn.Module):

    def __init__(self, lam=1):
        super(MultiTaskLoss, self).__init__()
        self.lam = lam
        # L_cls使用交叉熵损失
        self.cls = nn.CrossEntropyLoss()
        # L_loc自定义
        self.loc = SmoothL1Loss()

    def forward(self, scores, preds, targets):
        """
        计算多任务损失。N表示RoI数目
        :param scores: [N, C]，C表示类别数
        :param preds: [N, 4]，4表示边界框坐标x,y,w,h
        :param targets: [N]，0表示背景类
        :return:
        """
        N = targets.shape[0]
        return self.cls(scores, targets) + self.loc(scores[range(N), self.indicator(targets)],
                                                    preds[range(N), self.indicator(preds)])

    def indicator(self, cate):
        return cate != 0


roi_pool.py


In [3]:
import torch
import torch.nn as nn

class ROI_Pool(nn.Module):

  def __init__(self, size):
    super(ROI_Pool, self).__init__()
    assert len(size) == 2, 'size参数输入（长宽）'
    pool_func = nn.AdaptiveMaxPool2d

    self.roi_pool = pool_func(size)

  def forward(self, feature_maps):
    assert feature_maps.dim() == 4, 'Expected 4d input of (N, C, H, W)'
    return self.roi_pool(feature_maps)

    

smooth_l1_loss


In [ ]:

import torch
import torch.nn as nn


class SmoothL1Loss(nn.Module):

    def __init__(self):
        super(SmoothL1Loss, self).__init__()

    def forward(self, preds, targets):
        """
        计算边界框回归损失。N表示RoI个数
        :param preds: 大小为[N, 4]，分别保存每个边界框的预测x,y,w,h
        :param targets: 大小为[N, 4]，分别保存每个边界框的实际x,y,w,h
        :return:
        """
        res = self.smooth_l1(preds - targets)
        return torch.sum(res)

    def smooth_l1(self, x):
        if torch.abs(x) < 1:
            return 0.5 * torch.pow(x, 2)
        else:
            return torch.abs(x) - 0.5


vgg16_roi.py


In [2]:
# -*- coding: utf-8 -*-

"""
@date: 2020/3/31 下午2:55
@file: vgg16_roi.py
@author: zj
@description: 
"""

import torch
import torch.nn as nn
import torchvision.models as models

import models.roi_pool as roi_pool


class VGG16_RoI(nn.Module):

    def __init__(self, num_classes=1000, init_weights=True):
        """
        :param num_classes: 类别数，不包括背景类别
        :param init_weights:
        """
        super(VGG16_RoI, self).__init__()
        # VGG16模型的卷积层设置，取消最后一个最大池化层'M'
        feature_list = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512]

        self.features = models.vgg.make_layers(feature_list)
        # self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.roipool = roi_pool.ROI_Pool((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            # nn.Linear(4096, num_classes),
        )
        self.softmax = nn.Linear(4096, num_classes + 1)
        self.bbox = nn.Linear(4096, num_classes * 4)

        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        # x = self.avgpool(x)
        x = self.roipool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        classify = self.softmax(x)
        regression = self.bbox(x)
        return classify, regression

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


if __name__ == '__main__':
    # model = models.vgg16(pretrained=True)
    model = VGG16_RoI()
    print(model)


SyntaxError: ignored